In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import cv2
import os
import glob
import random


In [3]:
!pip install sklearn-som
from sklearn_som.som import SOM


In [4]:
from collections import  defaultdict
def win_map(som, data, return_indices=False):
        """Returns a dictionary wm where wm[(i,j)] is a list with:
        - all the patterns that have been mapped to the position (i,j),
          if return_indices=False (default)
        - all indices of the elements that have been mapped to the
          position (i,j) if return_indices=True"""
        winmap = defaultdict(list)
        for i, x in enumerate(data):
            winmap[som._find_bmu(x)].append(i if return_indices else x)
        return winmap

def get_labeled_neurons(labeled_vectors, emo_target, emptyDic, somap):
    dic = win_map(somap, labeled_vectors,return_indices=True )
    emotion_targets = emo_target
    counters = []
    for key in dic:
        space = np.zeros(len(dic[key]), dtype = int)
        for i in range(0, len(dic[key])):
            val = dic[key]
            space[i] = emotion_targets[val[i]]
        counters.append(space)
    for cnt, key in enumerate(dic):
        counts = np.bincount(counters[cnt])
        #print(counts)
        emptyDic[key] = np.argmax(counts)
    return 0

import matplotlib.lines as mlines
import matplotlib.pyplot as plt
%matplotlib inline



def get_labeledDic(somap, b_vectors_labeled, target):
  emptyDic = {}
  test_en = get_labeled_neurons(b_vectors_labeled, target, emptyDic, somap)

  lenn = somap.n *somap.m
  matrix_for_com = np.full(lenn, 8, dtype = int)

  for key in emptyDic.keys():
    matrix_for_com[key] = emptyDic[key]
  matrix_for_com = np.reshape(matrix_for_com, (somap.n,somap.m))



  location = somap._get_locations(somap.n,somap.m)
  newDic ={}
  for loc in location:
    if matrix_for_com[loc[0]][loc[1]] != 8:
      newDic[tuple(loc)] = matrix_for_com[loc[0]][loc[1]]

  return newDic



def fast_norm(x):
    """Returns norm-2 of a 1-D numpy array.
    """
    return np.sqrt(np.dot(x, x.T))


In [5]:
 def distance_map(som, reshapeweig, scaling='sum'):
        """Returns the distance map of the weights.
        If scaling is 'sum' (default), each cell is the normalised sum of
        the distances between a neuron and its neighbours. Note that this
        method uses the euclidean distance.

        Parameters
        ----------
        scaling : string (default='sum')
            If set to 'mean', each cell will be the normalized
            by the average of the distances of the neighbours.
            If set to 'sum', the normalization is done
            by the sum of the distances.
        """
        weights = reshapeweig
        if scaling not in ['sum', 'mean']:
            raise ValueError(f'scaling should be either "sum" or "mean" ('
                             f'"{scaling}" not valid)')

        um = np.nan * np.zeros((weights.shape[0],
                          weights.shape[1],
                          8))  # 2 spots more for hexagonal topology

        ii = [[0, -1, -1, -1, 0, 1, 1, 1]]*2
        jj = [[-1, -1, 0, 1, 1, 1, 0, -1]]*2


        for x in range(weights.shape[0]):
            for y in range(weights.shape[1]):
                w_2 = weights[x, y]
                e = y % 2 == 0 #only used in hexagonal
                for k, (i, j) in enumerate(zip(ii[e], jj[e])):
                    if (x+i >= 0 and x+i < weights.shape[0] and
                            y+j >= 0 and y+j < weights.shape[1]):
                        w_1 = weights[x+i, y+j]
                        um[x, y, k] = fast_norm(w_2-w_1)

        if scaling == 'mean':
            um = np.nanmean(um, axis=2)
        if scaling == 'sum':
            um = np.nansum(um, axis=2)

        return um/um.max()



In [6]:
def draw_labeled_neurons_map(somap, MyDic,savepath, formato):
    plt.figure(figsize=(7,7))
    m, n = somap.shape
    stanrweig = somap.weights
    reshapeweig=stanrweig.reshape((m,n,somap.dim))

    plt.pcolor(distance_map(somap, reshapeweig), cmap='bone_r')  # plotting the distance map as background
    plt.colorbar()

    ## Plotting the response for each pattern in the iris dataset
    ## different colors and markers for each label
    blue_star = mlines.Line2D([], [], color='blue', marker='o', linestyle='None',
                          markersize=10, label='Blue stars')
    red_square = mlines.Line2D([], [], color='orange', marker='v', linestyle='None',
                          markersize=10, label='Red squares')
    purple_triangle = mlines.Line2D([], [], color='green', marker='x', linestyle='None',
                          markersize=10, label='Purple triangles')
    green_star = mlines.Line2D([], [], color='red', marker='p', linestyle='None',
                          markersize=10, label='Blue stars')
    orange_square = mlines.Line2D([], [], color='purple', marker='P', linestyle='None',
                          markersize=10, label='Red squares')
    yellow_triangle = mlines.Line2D([], [], color='brown', marker='s', linestyle='None',
                          markersize=10, label='Purple triangles')
    black_triangle = mlines.Line2D([], [], color='pink', marker='D', linestyle='None',
                          markersize=10, label='Purple triangles')

    markersleg = [black_triangle, blue_star, red_square, purple_triangle, green_star, orange_square,yellow_triangle   ]

    #markers = [1,2,3,4,5,6,7]
    markers = ['D', 'o','v', 'x', 'p', 'P','s' ]
    colors = ['C6','C0', 'C1', 'C2', 'C3', 'C4','C5']
    lala = ['Neutral','Anger', 'Disgust', 'Fear', 'Happy', 'Sadness', 'Surprise' ]
    for key in MyDic:
        # palce a marker on the winning position for the sample xx
        w = key
        sign = MyDic[key]

        if sign > 2:#no hay labels 2 porque las filtramos al hacer la base de dates
            sign = sign -1
        plt.plot(w[1]+.5, w[0]+.5, markers[sign], markerfacecolor='None',
                 markeredgecolor=colors[sign], markersize=12, markeredgewidth=2)


    lgd = plt.legend(handles = markersleg, labels = lala, bbox_to_anchor=(0, 1), loc='upper right', ncol=1)

    plt.title('Clasificación del Expression Map')
    #b_tuple = tuple(map(tuple, b_vector))
    plt.savefig(savepath,  bbox_extra_artists=(lgd,), bbox_inches='tight',format=formato)
    plt.show()
    return 0

In [12]:
def get_error_new(b_vect_label, lab_target, labeled_neurons,  e_som):
  correct = 0
  for i in range(0, len(b_vect_label)):
    label = get_label(b_vect_label[i], labeled_neurons, e_som)
    actual_label = lab_target[i]
    if label == actual_label:
      correct = correct + 1
  return correct, len(b_vect_label), correct/len(b_vect_label)

def get_error_new_average( cantidad, b_vect_label, lab_target, labeled_neurons, e_som):
  error_array = np.zeros(cantidad)
  for i in range(0, cantidad):
    _ , _ , acc = get_error_new(b_vect_label, lab_target, labeled_neurons, e_som)
    error_array[i] = acc
  return np.sum(error_array)/len(error_array)* cantidad



def get_label(instance, labeled_neurons,  som):
  win_neuron = som._find_bmu(instance)
  if win_neuron in labeled_neurons.keys():
    return labeled_neurons[win_neuron]
  else:
    index_neu = get_close_neurons(win_neuron, som)
    rank = []
    for i in range(0, len(index_neu)):
      if( index_neu[i] in labeled_neurons.keys() ):
        rank.append(labeled_neurons[index_neu[i]])
    rank = np.asarray(rank)
    counts = np.bincount(rank)
    return np.argmax(counts)



def get_close_neurons(win_neuron, som):

  m = som.m
  n = som.n

  row = int( win_neuron /m)
  col = int(win_neuron % n)
  ref_mat = np.zeros((n,m))

  if row == 0 and col == 0:
    right = win_neuron +1
    up = win_neuron + n
    up_right = win_neuron + n+ 1
    return [right, up, up_right]
  elif col == 0 and row < n-1:
    right = win_neuron +1
    up = win_neuron + n
    up_right = win_neuron + n+ 1
    down= win_neuron - n
    down_right = win_neuron -n + 1
    return [right, up, up_right, down, down_right]
  elif col == 0 and row == n-1:
    right = win_neuron +1
    down= win_neuron - n
    down_right = win_neuron -n + 1
    return [right, down, down_right]
  elif  col < n-1 and row == n-1:
    right = win_neuron +1
    down= win_neuron - n
    down_right = win_neuron -n + 1
    down_left = win_neuron - n - 1
    left =     win_neuron -1
    return [right,  down, down_right, down_left, left]
  elif col == n-1 and row == n-1:
    down= win_neuron - n
    down_left = win_neuron - n - 1
    left =     win_neuron -1
    return [ down, down_left, left]
  elif col == n-1 and row > 0:
    left = win_neuron  - 1
    up = win_neuron + n
    up_left = win_neuron + n - 1
    down= win_neuron - n
    down_right = win_neuron -n + 1
    return [left,  up, up_left, down, down_right]
  elif col == n-1 and row == 0:
    up = win_neuron + n
    up_left = win_neuron + n - 1
    left= win_neuron - 1
    return [ up, up_left, left]
  elif col < n-1 and row == 0:
    right = win_neuron +1
    up = win_neuron + n
    up_right = win_neuron + n+ 1
    up_left= win_neuron + n -1
    left = win_neuron -1
    return [right, up, up_right, up_left, left]
  else:
    right = win_neuron +1
    up = win_neuron + n
    up_right = win_neuron + n+ 1
    up_left = win_neuron +n -1
    down= win_neuron - n
    down_right = win_neuron -n + 1
    down_left = win_neuron - n -1
    left = win_neuron -1
    return [right, up, up_right, up_left, down, down_right, down_left, left]



def get_error_with_parameters2(lera, sig, ni, mi):
  accuracy = np.zeros(10)
  for i in range(0,10):
    aa_som = SOM(m=mi, n=ni, dim=392,lr=lera, sigma=sig,  max_iter=100500)
    aa_som.fit(b_vector_AV2)
    dicto = {}
    dicto = get_labeledDic(aa_som, b_vector_label_AV2, target_AV2)
    nueDic = {}
    get_labeled_neurons(b_vector_AV2, target_AV2, nueDic, aa_som)
    correct, total, acc = get_error_new( b_vector_AV2, target_AV2,nueDic, aa_som)
    accuracy[i] = acc
  return accuracy


def get_error_with_parameters(lera, sig, ni, mi, epo= 1):
  accuracy = np.zeros(10)
  for i in range(0,10):
    aa_som = SOM(m=mi, n=ni, dim=492,lr=lera, sigma=sig,  max_iter=100500)
    aa_som.fit(b_vectors_labeled_fs, epochs= epo)
    dic_v3 = win_map(aa_som, b_vectors_labeled_fs, return_indices=True)
    len(dic_v3.keys())
    dicto = {}
    dicto = get_labeledDic(aa_som, b_vectors_labeled_fs, target_fs)
    nueDic = {}
    get_labeled_neurons(b_vectors_labeled_fs, target_fs, nueDic, aa_som)
    correct, total, acc = get_error_new( b_vectors_labeled_fs, target_fs,nueDic, aa_som)
    accuracy[i] = acc
  return accuracy



def get_error_with_parameters_test(lera, sig, ni, mi, epo=1):
  accuracy = np.zeros(10)
  for i in range(0,10):
    aa_som = SOM(m=mi, n=ni, dim=492,lr=lera, sigma=sig,  max_iter=100500)
    aa_som.fit(b_vector_aux,  epochs= epo)
    dicto = {}
    dicto = get_labeledDic(aa_som, b_vectors_labeled_fs, target_fs)
    nueDic = {}
    get_labeled_neurons(b_vector_aux, target_fs, nueDic, aa_som)
    correct, total, acc = get_error_new( b_vector_test_total_fs, test_emotions_total,nueDic, aa_som)
    accuracy[i] = acc
  return accuracy



















In [9]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def get_prediction(b_vect_label, lab_target, labeled_neurons,  e_som):
  labels = []
  for i in range(0, len(b_vect_label)):
    label = get_label(b_vect_label[i], labeled_neurons, e_som)
    labels.append(label)
  return labels


#sns.heatmap(confusion_matrix,
#            annot=True,
#            fmt='g',
#            xticklabels=['Neutral','Anger','Disgust','Fear','Happy','Sadness','Surprise'],
#            yticklabels=['Neutral','Anger','Disgust','Fear','Happy','Sadness','Surprise'])

# display matrix
#plt.ylabel('Actual',fontsize=12)
#plt.xlabel('Prediction',fontsize=12)
#plt.show()